# Air Properties

Aerodynamics is the study of objects moving through a fluid we call air. In our normal lives, we hardly ever think much about air, except when the winds are blowing hard enough to cause problems. In aviation, air is the medium through which we are able to fly, so it is important to understand how that air works. In this section we will look at some of the basic properties of air, since we will be working with those properties in our analyses.

If you pay attention to weather forecasts, you know that the basic properties vary wildly depending on where you live. For that reason, researchers have developed a standard model for our atmosphere that researchers use in their work. The model we will use is the *1976 U.S. Standard Atmosphere* {cite}`stdatm`. The Python **fluids** package implements this model. We will use that together with **pint** to add unints to the numbers.

In [79]:
from pint import _DEFAULT_REGISTRY as u

In [80]:
atm = fluids.units.ATMOSPHERE_1976(Z=0*u.m)

The parameter provided specifies the height above sea level we want to use. In this example, we will be working at sea level.

### Basic Properties

Here are the properties we can retrieve from this *Standard Atmosphere* model:

In [81]:
air_properties = { 
    "T_inf": atm.T,
    "P_inf": atm.P,
    "rho_inf": atm.rho,
    "k_inf": atm.k,
    "Gravity": atm.g,
    "mu_inf": atm.mu,
    "a_inf": atm.v_sonic,
    "nu_inf": mu_inf/rho_inf
}

for key in air_properties:
    print(key,":",air_properties[key])

T_inf : 288.15 kelvin
P_inf : 101325.0 pascal
rho_inf : 1.2249991558877125 kilogram / meter ** 3
k_inf : 0.02532588426426395 watt / kelvin / meter
Gravity : 9.80665 meter / second ** 2
mu_inf : 1.789380278077583e-05 pascal * second
a_inf : 340.2941077869353 meter / second
nu_inf : 1.4607196008889362e-05 meter ** 3 * pascal * second / kilogram


In [ ]:
### Gas Constant

In the standard model, air is a gas that obeys the *Perfect Gas Law*:

\begin{equation}
P = \rho R_{specific} T
\end{equation}

Here, $R_{specific}$ is the gas constant for air. We will calculate this number using the data values from the standard atmosphere:

In [82]:
P_inf = air_properties['P_inf']
rho_inf = air_properties['rho_inf']
T_inf = air_properties['T_inf']

R_s = P_inf / (rho_inf * T_inf)
R_s.to('ft * lbf/(slug degR)')

1716.5630846421818 <Unit('foot * force_pound / degree_Rankine / slug')>

In [83]:
R_s.to_base_units()

287.05307204706463 <Unit('meter ** 2 / kelvin / second ** 2')>

As a check, the *Universal Gas Constant is defined as:

\begin{equation}
R = R_s M
\end{equation}

Where $M$ is the molar mass of air:

In [87]:
M = 28.9645 * u.grams/u.mole

R = R_s*M
R.to_base_units()

8.314348705307204 <Unit('kilogram * meter ** 2 / kelvin / mole / second ** 2')>

This is not an exact match with values found on-line, but it is close.

### Specific Heat Constants

The *specific heat capacity* for constant volume $c_v$ is a measure of the amount of heat that must be added to one unit of mass to raise the temperature one degree. A related term, the *specific heat capacity for constant pressure* $c_p$.

For an ideal gas, these terms are related as follows:

\begin{equation}
c_p - c_v = R
\end{equation}

Where $R$ is the ideal gas constant.

Another common term is the *heat capacity ratio* $\gamma$ defined as

\begin{equation}
\gamma = \frac{c_p}{c_v} \approx 1.4
\end{equation}

In [40]:
c_p = 1.005 * u.kJ / (u.kg*u.degK)

In [41]:
c_p.to_base_units()

1004.9999999999999 <Unit('meter ** 2 / kelvin / second ** 2')>

In [42]:
c_p.to('ft**2/(sec**2 * degR)')

6009.849982662929 <Unit('foot ** 2 / degree_Rankine / second ** 2')>

In [43]:
c_v = 0.718 * u.kJ / (u.kg * u.degK)
c_v.to_base_units()

718.0 <Unit('meter ** 2 / kelvin / second ** 2')>

In [44]:
c_v.to('ft**2/(sec**2 * degR)')

4293.604266220878 <Unit('foot ** 2 / degree_Rankine / second ** 2')>

In [45]:
gamma = c_p/c_v
gamma

1.3997214484679665 <Unit('dimensionless')>

In [ ]:
As a check, we can calculate the gas consant using these values as follows:


In [88]:
Rcheck = c_p - c_v
Rcheck.to_base_units()

286.99999999999994 <Unit('meter ** 2 / kelvin / second ** 2')>

In [27]:
a = (1.4 * R_s * T_inf)**0.5
a.to('ft/sec')

1116.4504848652732 <Unit('foot / second')>

In [ ]:
Again, not an exact match, but close!

### Prandtl Number

\begin{equation}
Pr = \frac{c_p \mu}{k}
\end{equation}

In [74]:
Pr_inf = c_p * mu_inf / k_inf
Pr_inf.to_base_units()

0.7100747838469347 <Unit('dimensionless')>

## Isentropic Flow

For isentropic flow, we have several useful equations we can use:

### Speed of Sound

The standard model provides a speed of sound value. We can verify that value by using this equation:

\begin{equation}
a = \sqrt{\gamma R T} = 
\end{equation}

In [91]:
a = (gamma * R_s * T_inf)**0.5
a.to_base_units()

340.26025272961755 <Unit('meter / second')>

\begin{equation}
M = \frac{v}{a}
\end{equation}
\begin{equation}
\frac{p}{p_t} = (1 + \frac{\gamma - 1}{2}M^2)^{\frac{-\gamma}{\gamma - 1}}
\end{equation}

\begin{equation}
q = \frac{1}{2}\rho v^2
\end{equation}

The original version of the **rrbaxi** test program used data from an AEDC wind tunnel test of an ogive-cylinder. Using th Mac number, let's get a velocity:

In [121]:
M_inf = 3.5 # test case
U_inf = M_inf * a
U_inf.to('ft/sec')

3907.187941449021 <Unit('foot / second')>

### Reynolds Number

\begin{equation}
Re = \frac{\rho u L}{\mu}
\end{equation}

From the properties defined so far, we can calculate the free stream Reynolds Number:

In [122]:
L = 50 * u.inches
Re_inf = rho_inf * U_inf * L / mu_inf
Re_inf.to_base_units()

103541899.65430248 <Unit('dimensionless')>

### Reference Properties

The following wing reference properties are used to non-dimensionalize the governing equations:


#### Velocities

\begin{equation}
U_{ref} = U_{\infty}
\end{equation}

In [96]:
U_ref = U_inf
U_ref.to_base_units()

2024.5485037412245 <Unit('meter / second')>

In [116]:
U_sf = (U_inf/U_ref).to_base_units()
U_sf

1.0 <Unit('dimensionless')>

#### Temperature

\begin{equation}
T_{ref} = U_{ref}^2 / c_p
\end{equation}

In [98]:
T_ref = U_ref**2/c_p
T_ref.to_base_units()

4078.4046208963505 <Unit('kelvin')>

In [117]:
T_sf = (T_inf/T_ref).to_base_units()
T_sf

0.07065262689327535 <Unit('dimensionless')>

#### Density

\begin{equation}
\rho_{ref} = \rho_\infty
\end{equation}

In [99]:
rho_ref = rho_inf
rho_ref.to_base_units()

1.2249991558877125 <Unit('kilogram / meter ** 3')>

In [118]:
rho_sf = (rho_inf/rho_ref).to_base_units()
rho_sf

1.0 <Unit('dimensionless')>

In [ ]:
#### Dynamic Viscosity

\begin{equation}
\mu_{ref} = \mu_{inf} * T_{ref}
\end{equation}

In [120]:
mu_inf.to_base_units()

1.789380278077583e-05 <Unit('kilogram / meter / second')>

In [114]:
mu_ref = c_p * mu_inf / k_inf
mu_ref.to_base_units()

0.7100747838469347 <Unit('dimensionless')>

In [119]:
mu_sf = (mu_inf/mu_ref).to_base_units()
mu_sf

2.5199884840063633e-05 <Unit('kilogram / meter / second')>

\begin{equation}
k_{ref} = k_{\infty} * T_{ref}
\end{equation}

In [110]:
k_ref = c_p * mu_ref/Pr_inf
k_ref.to_base_units()

1004.9999999999999 <Unit('meter ** 2 / kelvin / second ** 2')>

In [111]:
(k_inf/k_ref).to_base_units()

2.5199884840063637e-05 <Unit('kilogram / meter / second')>

\begin{equation}
H_{ref} = U_{ref}^2
\end{equation}

In [102]:
H_ref = U_ref**2
H_ref.to_base_units()

4098796.644000832 <Unit('meter ** 2 / second ** 2')>


\begin{equation}
Pr = c_p * \mu_{\infty} / k_\infty
\end{equation}

In [103]:
Re_ref = rho_ref * U_ref * L / mu_ref
Re_ref.to_base_units()

4435.714710596741 <Unit('kilogram / meter / second')>

In [104]:
Re_ref.to('lb/(ft*sec)')

2980.6626680909258 <Unit('pound / foot / second')>

In [47]:
Test_Re_ref = 2179168
L = 50 * u.inches

Given M_inf, P_o_inf, T_inf, gamma, c_p, Pr_inf, R_inf

p_inf = P_o_inf *( 1 + (gamma-1)/2 M_inf^2)^(-gamma/(gamma-1))

rho_inf = P_inf/(R_inf I *T_inf)

a_inf = sqrt(gamma * R_inf * T_inf)

U_inf = M_inf ^2/c_p_inf

mu_ref = 2.270 * T_ref^3/2 / (T_ref + 198.6) x 10-8

k_ref = c_p_inf * mu_ref / Pr_inf

H_inf = (1 + 2/(gamma-1) M_inf^2))/2

p_inf = 1/(gamma*M_inf^2)

P_o_inf = p_inf + 1/2 rho U_inf **2/g

In [65]:
P_o_inf = P_inf / (1 + (gamma-1)/2 * Mach**2)**(-gamma/(gamma-1))
P_o_inf

152195941.21202007 <Unit('pascal')>

In [67]:
q = 1/2 * rho_inf * U_inf ** 2
q

2511010.81875 <Unit('pascal')>

In [69]:
p_o_inf = P_inf + q

In [70]:
p_o_inf

2612335.81875 <Unit('pascal')>

In [71]:
M_inf = Mach
p_o_inf *( 1 + (gamma-1)/2*M_inf**2)**(-gamma/(gamma-1))

1739.172048393225 <Unit('pascal')>